In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import math

from os import listdir
from os.path import isfile, join

In [2]:
def aggregate_range(minLon, maxLon, minLat, maxLat, df):
    data = df.loc[(df['lon'] >= minLon) & (df['lon'] < maxLon) & (df['lat'] > minLat) & (df['lat'] <= maxLat)]
    if data.empty:
        return data
    
    data_agg = pd.DataFrame(data.apply({'Calories 2050': np.sum, 
                                        'ΔCalories': np.sum,
                                        'Population 2050': np.sum})).T
    
    data_agg['min_lon'] = np.min(data.lon)
    data_agg['min_lat'] = np.min(data.lat)
    data_agg['max_lon'] = np.max(data.lon)
    data_agg['max_lat'] = np.max(data.lat)
    
    data_agg['percent_change_in_production'] = ((data_agg['Calories 2050'] / (data_agg['Calories 2050'] - data_agg['ΔCalories'])) - 1.0) * 100
    
    if np.abs(np.min(data.lon) - minLon) < 0.1:
        data_agg['min_lon'] = minLon - 0.025
    if np.abs(np.min(data.lat) - minLat) < 0.1:
        data_agg['min_lat'] = minLat - 0.025
    if np.abs(np.max(data.lon) - maxLon) < 0.1:
        data_agg['max_lon'] = maxLon + 0.025
    if np.abs(np.max(data.lat) - maxLat) < 0.1:
        data_agg['max_lat'] = maxLat + 0.025
    
    return data_agg

In [3]:
def create_bins(nb_bins, scope):
    nb_lat = np.sqrt(nb_bins / 2)
    nb_lon = 2 * nb_lat
    nb_lon = int(np.round(nb_lon))
    nb_lat = int(np.round(nb_lat))
    
    lon_bins = np.linspace(scope[0], scope[1], num=nb_lon)
    lat_bins = np.linspace(scope[2], scope[3], num=nb_lat)
    
    longitude_bins = []
    for first, second in zip(lon_bins[:-1],lon_bins[1:]):
        longitude_bins.append((first, second))
        
    latitude_bins = []
    for first, second in zip(lat_bins[:-1],lat_bins[1:]):
        latitude_bins.append((first, second))
    
    return (longitude_bins, latitude_bins)

In [4]:
def aggregate_data(nb_bins, data, scope=(-180, 180, -90, 90)):
    df = pd.DataFrame([], columns=['Calories 2050', 'ΔCalories', 'Population 2050', 'min_lon', 'min_lat',
                                   'max_lon', 'max_lat', 'percent_change_in_production'])
    
    (lon_bins, lat_bins) = create_bins(nb_bins, scope)
    
    for (minLon, maxLon) in lon_bins:
        for (minLat, maxLat) in lat_bins:
            tmp = aggregate_range(minLon, maxLon, minLat, maxLat, data)
            if not tmp.empty:
                df = df.append(tmp)
    
    return df

In [5]:
ssps = ['SSP1', 'SSP2', 'SSP3', 'SSP4', 'SSP5']

In [7]:
%%time
for name in ssps:
    print('Reading data/2050/' + name + '_country.csv')
    data = pd.read_csv('data/2050/' + name + '_country.csv')
    print('Processing data/2050/' + name + '_World.csv')
    filtered = data.dropna()
    data_scope = (np.min(filtered['lon']), np.max(filtered['lon']) + 0.01, np.min(filtered['lat']), np.max(filtered['lat']) + 0.01)
    df = aggregate_data(17500, filtered, scope=data_scope)
    print('Writing data/2050/' + name + '_World.csv')
    df.to_csv('data/2050/' + name + '_World.csv')

Reading data/2050/SSP1_country.csv
Processing data/2050/SSP1_World.csv
Writing data/2050/SSP1_World.csv
Reading data/2050/SSP2_country.csv
Processing data/2050/SSP2_World.csv
Writing data/2050/SSP2_World.csv
Reading data/2050/SSP3_country.csv
Processing data/2050/SSP3_World.csv
Writing data/2050/SSP3_World.csv
Reading data/2050/SSP4_country.csv
Processing data/2050/SSP4_World.csv
Writing data/2050/SSP4_World.csv
Reading data/2050/SSP5_country.csv
Processing data/2050/SSP5_World.csv
Writing data/2050/SSP5_World.csv
Wall time: 35min 28s


In [30]:
%%time
for name in ssps:
    data = pd.read_csv('data/2050/' + name + '_country.csv')
    print('Reading data/2050/' + name + '_country.csv')
    for country in np.unique(data['country']):
        print('Generating data for', country)
        data_country = data.loc[(data['country'] == country)]
        data_scope = (np.min(data_country['lon']), np.max(data_country['lon']) + 0.01, np.min(data_country['lat']), np.max(data_country['lat']) + 0.01)
        nb_bins = np.min([len(data_country), 15000])
        df = aggregate_data(nb_bins, data_country, scope=data_scope)
        print('Writing data/2050/' + name + '_' + country + '.csv')
        df.to_csv('data/2050/' + name + '_' + country + '.csv')

Reading data/2050/SSP1_country.csv
Generating data for Afghanistan
Writing data/2050/SSP1_Afghanistan.csv
Generating data for Albania
Writing data/2050/SSP1_Albania.csv
Generating data for Algeria
Writing data/2050/SSP1_Algeria.csv
Generating data for Angola
Writing data/2050/SSP1_Angola.csv
Generating data for Argentina
Writing data/2050/SSP1_Argentina.csv
Generating data for Armenia
Writing data/2050/SSP1_Armenia.csv
Generating data for Australia
Writing data/2050/SSP1_Australia.csv
Generating data for Austria
Writing data/2050/SSP1_Austria.csv
Generating data for Azerbaijan
Writing data/2050/SSP1_Azerbaijan.csv
Generating data for Bangladesh
Writing data/2050/SSP1_Bangladesh.csv
Generating data for Belarus
Writing data/2050/SSP1_Belarus.csv
Generating data for Belgium
Writing data/2050/SSP1_Belgium.csv
Generating data for Belize
Writing data/2050/SSP1_Belize.csv
Generating data for Benin
Writing data/2050/SSP1_Benin.csv
Generating data for Bhutan
Writing data/2050/SSP1_Bhutan.csv
Ge

Writing data/2050/SSP1_Republic of Serbia.csv
Generating data for Republic of the Congo
Writing data/2050/SSP1_Republic of the Congo.csv
Generating data for Romania
Writing data/2050/SSP1_Romania.csv
Generating data for Russia
Writing data/2050/SSP1_Russia.csv
Generating data for Rwanda
Writing data/2050/SSP1_Rwanda.csv
Generating data for Saudi Arabia
Writing data/2050/SSP1_Saudi Arabia.csv
Generating data for Senegal
Writing data/2050/SSP1_Senegal.csv
Generating data for Sierra Leone
Writing data/2050/SSP1_Sierra Leone.csv
Generating data for Slovakia
Writing data/2050/SSP1_Slovakia.csv
Generating data for Slovenia
Writing data/2050/SSP1_Slovenia.csv
Generating data for Solomon Islands
Writing data/2050/SSP1_Solomon Islands.csv
Generating data for Somalia
Writing data/2050/SSP1_Somalia.csv
Generating data for South Africa
Writing data/2050/SSP1_South Africa.csv
Generating data for South Korea
Writing data/2050/SSP1_South Korea.csv
Generating data for South Sudan
Writing data/2050/SSP

Writing data/2050/SSP2_Israel.csv
Generating data for Italy
Writing data/2050/SSP2_Italy.csv
Generating data for Ivory Coast
Writing data/2050/SSP2_Ivory Coast.csv
Generating data for Jamaica
Writing data/2050/SSP2_Jamaica.csv
Generating data for Japan
Writing data/2050/SSP2_Japan.csv
Generating data for Jordan
Writing data/2050/SSP2_Jordan.csv
Generating data for Kazakhstan
Writing data/2050/SSP2_Kazakhstan.csv
Generating data for Kenya
Writing data/2050/SSP2_Kenya.csv
Generating data for Kosovo
Writing data/2050/SSP2_Kosovo.csv
Generating data for Kuwait
Writing data/2050/SSP2_Kuwait.csv
Generating data for Kyrgyzstan
Writing data/2050/SSP2_Kyrgyzstan.csv
Generating data for Laos
Writing data/2050/SSP2_Laos.csv
Generating data for Latvia
Writing data/2050/SSP2_Latvia.csv
Generating data for Lebanon
Writing data/2050/SSP2_Lebanon.csv
Generating data for Lesotho
Writing data/2050/SSP2_Lesotho.csv
Generating data for Liberia
Writing data/2050/SSP2_Liberia.csv
Generating data for Libya
W

Writing data/2050/SSP3_Burkina Faso.csv
Generating data for Burundi
Writing data/2050/SSP3_Burundi.csv
Generating data for Cambodia
Writing data/2050/SSP3_Cambodia.csv
Generating data for Cameroon
Writing data/2050/SSP3_Cameroon.csv
Generating data for Canada
Writing data/2050/SSP3_Canada.csv
Generating data for Central African Republic
Writing data/2050/SSP3_Central African Republic.csv
Generating data for Chad
Writing data/2050/SSP3_Chad.csv
Generating data for Chile
Writing data/2050/SSP3_Chile.csv
Generating data for China
Writing data/2050/SSP3_China.csv
Generating data for Colombia
Writing data/2050/SSP3_Colombia.csv
Generating data for Costa Rica
Writing data/2050/SSP3_Costa Rica.csv
Generating data for Croatia
Writing data/2050/SSP3_Croatia.csv
Generating data for Cuba
Writing data/2050/SSP3_Cuba.csv
Generating data for Cyprus
Writing data/2050/SSP3_Cyprus.csv
Generating data for Czech Republic
Writing data/2050/SSP3_Czech Republic.csv
Generating data for Democratic Republic of

Writing data/2050/SSP3_Switzerland.csv
Generating data for Syria
Writing data/2050/SSP3_Syria.csv
Generating data for Taiwan
Writing data/2050/SSP3_Taiwan.csv
Generating data for Tajikistan
Writing data/2050/SSP3_Tajikistan.csv
Generating data for Thailand
Writing data/2050/SSP3_Thailand.csv
Generating data for The Bahamas
Writing data/2050/SSP3_The Bahamas.csv
Generating data for Togo
Writing data/2050/SSP3_Togo.csv
Generating data for Trinidad and Tobago
Writing data/2050/SSP3_Trinidad and Tobago.csv
Generating data for Tunisia
Writing data/2050/SSP3_Tunisia.csv
Generating data for Turkey
Writing data/2050/SSP3_Turkey.csv
Generating data for Turkmenistan
Writing data/2050/SSP3_Turkmenistan.csv
Generating data for Uganda
Writing data/2050/SSP3_Uganda.csv
Generating data for Ukraine
Writing data/2050/SSP3_Ukraine.csv
Generating data for United Arab Emirates
Writing data/2050/SSP3_United Arab Emirates.csv
Generating data for United Kingdom
Writing data/2050/SSP3_United Kingdom.csv
Gener

Writing data/2050/SSP4_Mexico.csv
Generating data for Moldova
Writing data/2050/SSP4_Moldova.csv
Generating data for Mongolia
Writing data/2050/SSP4_Mongolia.csv
Generating data for Montenegro
Writing data/2050/SSP4_Montenegro.csv
Generating data for Morocco
Writing data/2050/SSP4_Morocco.csv
Generating data for Mozambique
Writing data/2050/SSP4_Mozambique.csv
Generating data for Myanmar
Writing data/2050/SSP4_Myanmar.csv
Generating data for Namibia
Writing data/2050/SSP4_Namibia.csv
Generating data for Nepal
Writing data/2050/SSP4_Nepal.csv
Generating data for Netherlands
Writing data/2050/SSP4_Netherlands.csv
Generating data for New Caledonia
Writing data/2050/SSP4_New Caledonia.csv
Generating data for New Zealand
Writing data/2050/SSP4_New Zealand.csv
Generating data for Nicaragua
Writing data/2050/SSP4_Nicaragua.csv
Generating data for Niger
Writing data/2050/SSP4_Niger.csv
Generating data for Nigeria
Writing data/2050/SSP4_Nigeria.csv
Generating data for North Korea
Writing data/2

Writing data/2050/SSP5_Equatorial Guinea.csv
Generating data for Eritrea
Writing data/2050/SSP5_Eritrea.csv
Generating data for Estonia
Writing data/2050/SSP5_Estonia.csv
Generating data for Ethiopia
Writing data/2050/SSP5_Ethiopia.csv
Generating data for Falkland Islands
Writing data/2050/SSP5_Falkland Islands.csv
Generating data for Fiji
Writing data/2050/SSP5_Fiji.csv
Generating data for Finland
Writing data/2050/SSP5_Finland.csv
Generating data for France
Writing data/2050/SSP5_France.csv
Generating data for French Guiana
Writing data/2050/SSP5_French Guiana.csv
Generating data for Gabon
Writing data/2050/SSP5_Gabon.csv
Generating data for Gambia
Writing data/2050/SSP5_Gambia.csv
Generating data for Georgia
Writing data/2050/SSP5_Georgia.csv
Generating data for Germany
Writing data/2050/SSP5_Germany.csv
Generating data for Ghana
Writing data/2050/SSP5_Ghana.csv
Generating data for Greece
Writing data/2050/SSP5_Greece.csv
Generating data for Guatemala
Writing data/2050/SSP5_Guatemal

Writing data/2050/SSP5_West Bank.csv
Generating data for Western Sahara
Writing data/2050/SSP5_Western Sahara.csv
Generating data for Yemen
Writing data/2050/SSP5_Yemen.csv
Generating data for Zambia
Writing data/2050/SSP5_Zambia.csv
Generating data for Zimbabwe
Writing data/2050/SSP5_Zimbabwe.csv
Wall time: 5h 53min 35s


In [21]:
for name in ssps:
    data = pd.read_csv('data/2050/' + name + '_country.csv')
    print('Reading data/2050/' + name + '_country.csv')
    for continent in np.unique(data['continent']):
        print('Generating data for', continent)
        data_continent = data.loc[(data['continent'] == continent)]
        data_scope = (np.min(data_continent['lon']), np.max(data_continent['lon']) + 0.01, np.min(data_continent['lat']), np.max(data_continent['lat']) + 0.01)
        nb_bins = np.min([len(data_continent), 17000])
        df = aggregate_data(nb_bins, data_continent, scope=data_scope)
        print('Writing data/2050/' + name + '_' + continent + '.csv')
        df.to_csv('data/2050/' + name + '_' + continent + '.csv')

Reading data/2050/SSP1_country.csv
Generating data for Africa
Writing data/2050/SSP1_Africa.csv
Generating data for Asia
Writing data/2050/SSP1_Asia.csv
Generating data for Europe
Writing data/2050/SSP1_Europe.csv
Generating data for North America
Writing data/2050/SSP1_North America.csv
Generating data for Oceania
Writing data/2050/SSP1_Oceania.csv
Generating data for South America
Writing data/2050/SSP1_South America.csv
Generating data for Unknown
Writing data/2050/SSP1_Unknown.csv
Reading data/2050/SSP2_country.csv
Generating data for Africa
Writing data/2050/SSP2_Africa.csv
Generating data for Asia
Writing data/2050/SSP2_Asia.csv
Generating data for Europe
Writing data/2050/SSP2_Europe.csv
Generating data for North America
Writing data/2050/SSP2_North America.csv
Generating data for Oceania
Writing data/2050/SSP2_Oceania.csv
Generating data for South America
Writing data/2050/SSP2_South America.csv
Generating data for Unknown
Writing data/2050/SSP2_Unknown.csv
Reading data/2050/SS

In [ ]:
#directory = 'data/2050/SSP5/'
#onlyfiles = [f for f in listdir(directory) if isfile(join(directory, f))]
#for file in onlyfiles:
#    data = pd.read_csv(directory + file)
#    data = data.drop(['Unnamed: 0', 'lat', 'lon', 'Δcal_per_ha', 'cal_per_ha 2050'], axis=1)
#    data['%change_in_production'] = ((data['Calories 2050'] / (data['Calories 2050'] - data['ΔCalories'])) - 1.0) * 100
#    data['%cropland 2050'] = data['%cropland 2050'] * 100
#    data.to_csv(directory + file)

In [18]:
#directory = 'data/2050/SSP5/'
#onlyfiles = [f for f in listdir(directory) if isfile(join(directory, f))]
#for file in onlyfiles:
#    data = pd.read_csv(directory + file)
#    data = data.rename(columns={'%cropland 2050': 'percent_cropland 2050', '%change_in_production': 'percent_change_in_production'})
#    data = data.drop(['Unnamed: 0'], axis=1)
#    data.to_csv(directory + file)